In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')

torch.manual_seed(1)
batch_size = 128
lr=0.01

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


device: cuda


In [3]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    train_loss = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)
 
    print('\n#Train set: Average loss: {:.4f}, Accuracy: {:.2f}%'.format(
        train_loss,
        100. * correct / len(train_loader.dataset)))      


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('#Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
class Net_0(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)               
        self.conv5 = nn.Conv2d(256, 512, 3)           #input - 7, Output 5, RF
        self.conv6 = nn.Conv2d(512, 1024, 3)          #input - 5, Output 3, RF
        self.conv7 = nn.Conv2d(1024, 10, 3)           #input - 3, Output 1, RF

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_0

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# 10: Test set: Average loss: 0.2577, Accuracy: 8935/10000 (89%)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pas

<ipython-input-5-b72feb762c6a>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [6]:
# Fix Relu before the last layer

class Net_1(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)               
        self.conv5 = nn.Conv2d(256, 512, 3)           #input - 7, Output 5, RF
        self.conv6 = nn.Conv2d(512, 1024, 3)          #input - 5, Output 3, RF
        self.conv7 = nn.Conv2d(1024, 10, 3)           #input - 3, Output 1, RF

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_1

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# 10: Test set: Average loss: 0.0281, Accuracy: 9916/10000 (99%)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pas

<ipython-input-6-97bb41efa4e8>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
# Replace FCs by 1x1x256x10 + GAP

class Net_2(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)                    

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_2

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# 10: Test set: Average loss: 0.0533, Accuracy: 9837/10000 (98%)
# 20: Test set: Average loss: 0.0283, Accuracy: 9900/10000 (99%)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7             [-1, 10, 7, 7]           2,570
         AvgPool2d-8             [-1, 10, 1, 1]               0
Total params: 390,410
Trainable params: 390,410
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.34
Params size (MB): 1.49
Estimated Total Size (MB): 

<ipython-input-7-004b74ddca23>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [8]:
# Remove conv4. max 64 ch

class Net_3(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     #input - 28, Output 28, RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)   #input - 14, Output 14, RF
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 10, 1)              #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)                    

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv3(x)))
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_3

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# 10: Test set: Average loss: 0.1315, Accuracy: 9608/10000 (96%)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          36,928
         MaxPool2d-5             [-1, 64, 7, 7]               0
            Conv2d-6             [-1, 10, 7, 7]             650
         AvgPool2d-7             [-1, 10, 1, 1]               0
Total params: 56,394
Trainable params: 56,394
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.79
Params size (MB): 0.22
Estimated Total Size (MB): 1.01
-------------------------------------------------------------

<ipython-input-8-1c5ab9dea573>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [9]:
# Add BN

class Net_4(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)   #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(64)
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 10, 1)              #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)                    

    def forward(self, x):
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
        x = self.pool2(F.relu(self.bn3(self.conv3(x))))
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_4

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

#epoch: 10
#loss=0.03565816953778267 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.78it/s]
#Test set: Average loss: 0.1311, Accuracy: 9607/10000 (96%)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
            Conv2d-3           [-1, 64, 28, 28]          18,496
       BatchNorm2d-4           [-1, 64, 28, 28]             128
         MaxPool2d-5           [-1, 64, 14, 14]               0
            Conv2d-6           [-1, 64, 14, 14]          36,928
       BatchNorm2d-7           [-1, 64, 14, 14]             128
         MaxPool2d-8             [-1, 64, 7, 7]               0
            Conv2d-9             [-1, 10, 7, 7]             650
        AvgPool2d-10             [-1, 10, 1, 1]               0
Total params: 56,714
Trainable params: 56,714
Non-trainable params: 0
-------------------------------------------------

<ipython-input-9-c3895eceecbc>:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [10]:
# Add BN to Net_2

class Net_5(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)                    

    def forward(self, x):
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
        x = self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x)))))))
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_5

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

#epoch: 10
#loss=0.019351879134774208 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.98it/s]
#Test set: Average loss: 0.0452, Accuracy: 9850/10000 (98%)

#epoch: 19
#loss=0.0030171473044902086 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 21.02it/s]
#Test set: Average loss: 0.0286, Accuracy: 9914/10000 (99%)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
            Conv2d-3           [-1, 64, 28, 28]          18,496
       BatchNorm2d-4           [-1, 64, 28, 28]             128
         MaxPool2d-5           [-1, 64, 14, 14]               0
            Conv2d-6          [-1, 128, 14, 14]          73,856
       BatchNorm2d-7          [-1, 128, 14, 14]             256
            Conv2d-8          [-1, 256, 14, 14]         295,168
       BatchNorm2d-9          [-1, 256, 14, 14]             512
        MaxPool2d-10            [-1, 256, 7, 7]               0
           Conv2d-11             [-1, 10, 7, 7]           2,570
        AvgPool2d-12             [-1, 10, 1, 1]        

<ipython-input-10-e21bbe09ed1e>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [11]:
# Add dropout

class Net_6(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.05)
        self.dropout2 = nn.Dropout(0.05)
        self.dropout3 = nn.Dropout(0.05)
        self.dropout4 = nn.Dropout(0.05)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_6

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


#epoch: 10
#loss=0.016568930819630623 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.46it/s]
#Test set: Average loss: 0.0456, Accuracy: 9866/10000 (99%)

#epoch: 20
#loss=0.008861390873789787 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.52it/s]
#Test set: Average loss: 0.0351, Accuracy: 9891/10000 (99%)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
       BatchNorm2d-5           [-1, 64, 28, 28]             128
           Dropout-6           [-1, 64, 28, 28]               0
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]          73,856
       BatchNorm2d-9          [-1, 128, 14, 14]             256
          Dropout-10          [-1, 128, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         295,168
      BatchNorm2d-12          [-1, 256, 14, 14]        

<ipython-input-11-c24d52b133e8>:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [12]:
# Larger dropout

class Net_7(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_7

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


#epoch: 10
#loss=0.007140759844332933 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.26it/s]
#Test set: Average loss: 0.0569, Accuracy: 9839/10000 (98%)

#epoch: 20
#loss=0.024968238547444344 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.36it/s]
#Test set: Average loss: 0.0469, Accuracy: 9878/10000 (99%)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
       BatchNorm2d-5           [-1, 64, 28, 28]             128
           Dropout-6           [-1, 64, 28, 28]               0
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]          73,856
       BatchNorm2d-9          [-1, 128, 14, 14]             256
          Dropout-10          [-1, 128, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         295,168
      BatchNorm2d-12          [-1, 256, 14, 14]        

<ipython-input-12-c908b5d6768b>:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [13]:
# Use cross_entropy instead of log_softmax

class Net_8(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        # return F.log_softmax(x)
        return x

Net = Net_8

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


from tqdm import tqdm
def train_ce(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test_ce(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
  
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# for epoch in range(1, 21):
#     print(f'epoch: {epoch}')
#     train_ce(model, device, train_loader, optimizer, epoch)
#     test_ce(model, device, test_loader)

#stopped early
# 59% accuracy at epoch 8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
       BatchNorm2d-5           [-1, 64, 28, 28]             128
           Dropout-6           [-1, 64, 28, 28]               0
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]          73,856
       BatchNorm2d-9          [-1, 128, 14, 14]             256
          Dropout-10          [-1, 128, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         295,168
      BatchNorm2d-12          [-1, 256, 14, 14]        

In [14]:
# Use cross_entropy instead of log_softmax, use Adam

class Net_9(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        # return F.log_softmax(x)
        return x

Net = Net_9

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


from tqdm import tqdm
def train_ce(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test_ce(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
  
optimizer = optim.Adam(model.parameters(), lr=0.01)

# for epoch in range(1, 21):
#     print(f'epoch: {epoch}')
#     train_ce(model, device, train_loader, optimizer, epoch)
#     test_ce(model, device, test_loader)

#stopped early
# 59% accuracy at epoch 8



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
       BatchNorm2d-5           [-1, 64, 28, 28]             128
           Dropout-6           [-1, 64, 28, 28]               0
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]          73,856
       BatchNorm2d-9          [-1, 128, 14, 14]             256
          Dropout-10          [-1, 128, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         295,168
      BatchNorm2d-12          [-1, 256, 14, 14]        

In [23]:
# Lighter model + Step LR

class Net_10(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        # self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        # self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(64)
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        # self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        # x = F.relu(self.bn2(self.conv2(x)))
        # x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        # x = F.relu(self.bn4(self.conv4(x)))
        # x = self.dropout4(x)

        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_10
lr = 0.01

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


#epoch: 10
# loss=0.19617058336734772 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.51it/s]
# Test set: Average loss: 0.2381, Accuracy: 9359/10000 (94%)


#epoch: 20
# 94%

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
         MaxPool2d-4           [-1, 32, 14, 14]               0
            Conv2d-5           [-1, 64, 14, 14]          18,496
       BatchNorm2d-6           [-1, 64, 14, 14]             128
           Dropout-7           [-1, 64, 14, 14]               0
         MaxPool2d-8             [-1, 64, 7, 7]               0
            Conv2d-9             [-1, 10, 7, 7]             650
        AvgPool2d-10             [-1, 10, 1, 1]               0
Total params: 19,658
Trainable params: 19,658
Non-trainable params: 0
-------------------------------------------------

<ipython-input-23-58b59ef5110d>:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [30]:
# Remove padding and have only 1 maxpool, reduce num params

class Net_11(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=0)   #input - 28, Output 26, RF
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 12, 3, padding=0)   #input - 26, Output 24, RF
        self.bn2 = nn.BatchNorm2d(12)
        self.conv3 = nn.Conv2d(12, 20, 3, padding=0)   #input - 24, Output 22, RF
        self.bn3 = nn.BatchNorm2d(20)

        # self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        # self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(20, 20, 3, padding=0) #input - 11, Output 9, RF
        self.bn4 = nn.BatchNorm2d(20)

        self.conv5 = nn.Conv2d(20, 10, 3, padding=0) #input - 9, Output 7, RF
        # self.bn3 = nn.BatchNorm2d(10)
        
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        # self.bn4 = nn.BatchNorm2d(256)
        # self.pool2 = nn.MaxPool2d(2, 2)

        # self.conv5 = nn.Conv2d(10, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)
        self.dropout5 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)

        x = self.pool1(x)

        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)
        # x = F.relu(self.bn5(self.conv5(x)))
        # x = self.dropout5(x)

        # x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_11
lr = 0.01

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))



#epoch: 20
#Train set: Average loss: 0.0003, Accuracy: 98.89%
#Test set: Average loss: 0.0384, Accuracy: 9875/10000 (99%)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
           Dropout-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 12, 24, 24]           1,092
       BatchNorm2d-5           [-1, 12, 24, 24]              24
           Dropout-6           [-1, 12, 24, 24]               0
            Conv2d-7           [-1, 20, 22, 22]           2,180
       BatchNorm2d-8           [-1, 20, 22, 22]              40
           Dropout-9           [-1, 20, 22, 22]               0
        MaxPool2d-10           [-1, 20, 11, 11]               0
           Conv2d-11             [-1, 20, 9, 9]           3,620
      BatchNorm2d-12             [-1, 20, 9, 9]        

<ipython-input-30-2a48d52867ab>:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [42]:
# Remove padding and have only 1 maxpool, increase num params to 15k

class Net_12(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=0)   #input - 28, Output 26, RF
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 10, 3, padding=0)   #input - 26, Output 24, RF
        self.bn2 = nn.BatchNorm2d(10)
        self.conv3 = nn.Conv2d(10, 24, 3, padding=0)   #input - 24, Output 22, RF
        self.bn3 = nn.BatchNorm2d(24)

        # self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        # self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(24, 32, 3, padding=0) #input - 11, Output 9, RF
        self.bn4 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 10, 3, padding=0) #input - 9, Output 7, RF
        # self.bn3 = nn.BatchNorm2d(10)
        
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        # self.bn4 = nn.BatchNorm2d(256)
        # self.pool2 = nn.MaxPool2d(2, 2)

        # self.conv5 = nn.Conv2d(10, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AdaptiveAvgPool2d((1, 1))          

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)
        self.dropout5 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)

        x = self.pool1(x)

        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)
        # x = F.relu(self.bn5(self.conv5(x)))
        # x = self.dropout5(x)

        # x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_12
lr = 0.01

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


#epoch: 10

#Train set: Average loss: 0.0004, Accuracy: 98.60%
#Test set: Average loss: 0.0691, Accuracy: 9774/10000 (97.74%)


#epoch: 20
#Train set: Average loss: 0.0003, Accuracy: 98.97%
#Test set: Average loss: 0.0437, Accuracy: 9861/10000 (98.61%)






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
           Dropout-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
           Dropout-6           [-1, 10, 24, 24]               0
            Conv2d-7           [-1, 24, 22, 22]           2,184
       BatchNorm2d-8           [-1, 24, 22, 22]              48
           Dropout-9           [-1, 24, 22, 22]               0
        MaxPool2d-10           [-1, 24, 11, 11]               0
           Conv2d-11             [-1, 32, 9, 9]           6,944
      BatchNorm2d-12             [-1, 32, 9, 9]        

<ipython-input-42-c2ab15f47960>:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:
# Add 10x10 after GAP

class Net_13(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=0)   #input - 28, Output 26, RF
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 10, 3, padding=0)   #input - 26, Output 24, RF
        self.bn2 = nn.BatchNorm2d(10)
        self.conv3 = nn.Conv2d(10, 24, 3, padding=0)   #input - 24, Output 22, RF
        self.bn3 = nn.BatchNorm2d(24)

        # self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        # self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(24, 32, 3, padding=0) #input - 11, Output 9, RF
        self.bn4 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 10, 3, padding=0) #input - 9, Output 7, RF
        # self.bn3 = nn.BatchNorm2d(10)
        
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        # self.bn4 = nn.BatchNorm2d(256)
        # self.pool2 = nn.MaxPool2d(2, 2)

        # self.conv5 = nn.Conv2d(10, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AdaptiveAvgPool2d((1, 1))          
        self.conv6 = nn.Conv2d(10, 10, 1)            #input - 7, Output 7, RF

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)
        self.dropout5 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)

        x = self.pool1(x)

        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)
        # x = F.relu(self.bn5(self.conv5(x)))
        # x = self.dropout5(x)

        # x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = self.conv6(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_13
lr = 0.01

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


#epoch: 10
#Train set: Average loss: 0.0004, Accuracy: 98.50%
#Test set: Average loss: 0.0859, Accuracy: 9737/10000 (97.37%)

#epoch: 20
#Train set: Average loss: 0.0003, Accuracy: 98.98%
#Test set: Average loss: 0.0546, Accuracy: 9827/10000 (98.27%)






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
           Dropout-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
           Dropout-6           [-1, 10, 24, 24]               0
            Conv2d-7           [-1, 24, 22, 22]           2,184
       BatchNorm2d-8           [-1, 24, 22, 22]              48
           Dropout-9           [-1, 24, 22, 22]               0
        MaxPool2d-10           [-1, 24, 11, 11]               0
           Conv2d-11             [-1, 32, 9, 9]           6,944
      BatchNorm2d-12             [-1, 32, 9, 9]        

<ipython-input-4-a7a2f685fa9e>:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [6]:
# lower pool layer loc

class Net_14(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=0)   #input - 28, Output 26, RF
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 10, 3, padding=0)   #input - 26, Output 24, RF
        self.bn2 = nn.BatchNorm2d(10)
        
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(10, 24, 3, padding=0)   #input - 24, Output 22, RF
        self.bn3 = nn.BatchNorm2d(24)

        # self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        # self.bn2 = nn.BatchNorm2d(64)
        
        
        self.conv4 = nn.Conv2d(24, 32, 3, padding=0) #input - 11, Output 9, RF
        self.bn4 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 10, 3, padding=0) #input - 9, Output 7, RF
        # self.bn3 = nn.BatchNorm2d(10)
        
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        # self.bn4 = nn.BatchNorm2d(256)
        # self.pool2 = nn.MaxPool2d(2, 2)

        # self.conv5 = nn.Conv2d(10, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AdaptiveAvgPool2d((1, 1))          
        self.conv6 = nn.Conv2d(10, 10, 1)            #input - 7, Output 7, RF

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)
        self.dropout5 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)
        
        x = self.pool1(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)

        

        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)
        # x = F.relu(self.bn5(self.conv5(x)))
        # x = self.dropout5(x)

        # x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = self.conv6(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_14
lr = 0.01

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


#epoch: 10
#Train set: Average loss: 0.0002, Accuracy: 99.25%
#Test set: Average loss: 0.0222, Accuracy: 9933/10000 (99.33%)

#epoch: 20
#Train set: Average loss: 0.0002, Accuracy: 99.40%
#Test set: Average loss: 0.0213, Accuracy: 9935/10000 (99.35%)






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
           Dropout-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
           Dropout-6           [-1, 10, 24, 24]               0
         MaxPool2d-7           [-1, 10, 12, 12]               0
            Conv2d-8           [-1, 24, 10, 10]           2,184
       BatchNorm2d-9           [-1, 24, 10, 10]              48
          Dropout-10           [-1, 24, 10, 10]               0
           Conv2d-11             [-1, 32, 8, 8]           6,944
      BatchNorm2d-12             [-1, 32, 8, 8]        

<ipython-input-6-3d4a0a1e3660>:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [8]:
from torch.optim.lr_scheduler import StepLR

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

for epoch in range(1, 26):
    print(f'#epoch: {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

#epoch: 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-6-3d4a0a1e3660>:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.04653191193938255 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.50it/s]


#Train set: Average loss: 0.0003, Accuracy: 98.85%


#Test set: Average loss: 0.0470, Accuracy: 9840/10000 (98.40%)

#epoch: 2


loss=0.022762542590498924 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.76it/s]

...


#Train set: Average loss: 0.0002, Accuracy: 99.40%


#Test set: Average loss: 0.0209, Accuracy: 9938/10000 (99.38%)

#epoch: 25


loss=0.00535220792517066 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.90it/s]


#Train set: Average loss: 0.0002, Accuracy: 99.38%


#Test set: Average loss: 0.0213, Accuracy: 9935/10000 (99.35%)

